In [1]:
# some useful mysklearn package import statements and reloads
import importlib, copy, os
from datetime import datetime

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

# uncomment once you paste your mypytable.py into mysklearn package
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MySimpleLinearRegressor, MyNaiveBayesClassifier, DecisionTreeClassifier, MyRandomForestClassifier, MyRandomClassifier, MyZeroRClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

In [2]:
print(1)
data = MyPyTable().load_from_file("input_data/tracks_features.csv")
print(2)
data.convert_to_numeric()
print(3)
# raw_data = copy.deepcopy(data.data)
# print(4)

1
2
3


In [3]:
# invert_col_names = True
# x_col_names = ["album_id", "artist_ids", "id", "release_date", "year", "name", "album", "disc_number"]
invert_col_names = False
x_col_names = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"]
y_col_name = "year" 
if invert_col_names:
        x_col_names = list(set(data.column_names).difference(set(x_col_names)))
        print(x_col_names)

In [4]:
data.apply_to_column("duration_ms", lambda x: [ y // 1000 for y in x ])


In [5]:
continuous_cols = ["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","duration_ms"]
myutils.bucketize(data.get_column(continuous_cols[0]))
for col_name in continuous_cols:
        data.apply_to_column(col_name, myutils.bucketize)

In [6]:
data.apply_to_column(y_col_name, myutils.music_years)

In [7]:
all_rows_x_data = myutils.prepare_x_list_from_mypytable(data, x_col_names)

In [8]:
x_data = all_rows_x_data[:]
y_data = data.get_column(y_col_name)[:]

In [9]:
print(y_data[:10])

[1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990]


In [10]:
learn = MyRandomForestClassifier(F=0.5, M=4, num_trees=20, max_threads=4)

In [11]:
print("starting fit")
s = datetime.now()
learn.fit(x_data, y_data)
e = datetime.now()
print("fit complete")
print("Duration:", e-s)


starting fit
Tree 0: Begin TDIDT
Tree 1: Begin TDIDT
Tree 2: Begin TDIDT
Tree 3: Begin TDIDT
Tree 3: TDIDT Complete
Tree 3: Begin Predict
Tree 1: TDIDT Complete
Tree 1: Begin Predict
Tree 0: TDIDT Complete
Tree 0: Begin Predict
Tree 3: Predict Complete
Tree 3: Begin Evaluate
Tree 3: Evaluate Complete. Accuracy: 0.8718217048469054, Precision: 0.1194741248379198, F-measure: 0.014727029116651316
Tree 1: Predict Complete
Tree 1: Begin Evaluate
Tree 1: Evaluate Complete. Accuracy: 0.8718881485947978, Precision: 0.12315091342204165, F-measure: 0.015435928950315404
Tree 0: Predict Complete
Tree 0: Begin Evaluate
Tree 0: Evaluate Complete. Accuracy: 0.8719258000519369, Precision: 0.12307761175463715, F-measure: 0.016267440607804374
Tree 4: Begin TDIDT
Tree 5: Begin TDIDT
Tree 6: Begin TDIDT
Tree 2: TDIDT Complete
Tree 2: Begin Predict
Tree 2: Predict Complete
Tree 2: Begin Evaluate
Tree 2: Evaluate Complete. Accuracy: 0.8719379814057171, Precision: 0.12297716875069173, F-measure: 0.01648131038

In [12]:
print([tree.accuracy for tree in learn.forest])
print([tree.precision for tree in learn.forest])

[0.8717945736498491, 0.8721500477010741, 0.8718482823460623, 0.8719086354170646, 0.871516617304499, 0.871564789021721, 0.872022143486381, 0.8718073087015286, 0.8720298952569685, 0.8721932361372041, 0.871761351775903, 0.8717142874544792, 0.8719285685414324, 0.8718089697952259, 0.8721301145767062, 0.8717807312023717, 0.8719717569775625, 0.8719994418725178, 0.8720669930162084, 0.8718881485947978]
[0.12391754062539932, 0.12380304541872178, 0.12368372547412632, 0.12367881652995694, 0.12366336837736755, 0.1236108458008138, 0.12360436677869571, 0.12359015149785438, 0.12355652441829812, 0.12352434326979572, 0.1233204660295671, 0.1233126883574905, 0.12327976798912108, 0.1232580423923148, 0.12325657785268586, 0.12324854782988304, 0.1232353887738746, 0.12318630233816667, 0.12316228008888161, 0.12315091342204165]


In [13]:
pstart = 0
pend = 1000000
predict_test = all_rows_x_data[pstart:pend]
predict_actual = y_data[pstart:pend]

In [14]:
y_test = learn.predict(predict_test)
print(y_test[:50])
print(predict_actual[:50])
mysklearn.myevaluation.metrics.accuracy(y_test, predict_actual)

[2010, 2010, 2010, 2010, 2010, 2010, 2010, 2000, 2010, 2010, 2010, 2010, 2010, 2000, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2000, 2010, 2010, 2010, 2010, 2010, 2010, 2000, 2010, 2000, 2010, 2010, 2010, 2000, 2010, 2010, 2000, 2010, 2010, 2010, 2010, 2010, 2010, 2000, 2010, 2000, 2000, 2010]
[1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 1990, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 1990, 1990, 1990]


0.8790664444444444

In [15]:
print(mysklearn.myevaluation.metrics.precision(y_test, predict_actual))
print(mysklearn.myevaluation.metrics.recall(y_test, predict_actual))
print(mysklearn.myevaluation.metrics.fmeasure(y_test, predict_actual))

0.3178754586112756
0.12797455217479783
0.020275861826473913


In [16]:
labels = sorted(set(y_data))
cmat = myevaluation.confusion_matrix(y_test, predict_actual, labels)
cmat_table = MyPyTable(data=cmat, column_names=labels)


In [17]:
cmat_table.pretty_print()

  1940    1950    1960    1970    1980    1990    2000    2010    2020
------  ------  ------  ------  ------  ------  ------  ------  ------
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       6       1       0       0       0       3       0
     0       0       0       4       1       4       4       1       0
     0       0       1       3      25      14      11       4       3
    13      25      73     193     514    2360    1388    1251      64
  1311    1946    4825    7587   11317   70041  166486  122860    4184
   297     732    2485    5218   10819   67154  215906  286902   13903
     0       0       1       0       0       7      21      16      16


In [18]:
bayes = MyNaiveBayesClassifier()

In [19]:
bayes.fit(x_data, y_data)

In [20]:
bayes_predict = bayes.predict(predict_test)

In [21]:
print(mysklearn.myevaluation.metrics.accuracy(bayes_predict, predict_actual))
print(mysklearn.myevaluation.metrics.precision(bayes_predict, predict_actual))
print(mysklearn.myevaluation.metrics.recall(bayes_predict, predict_actual))
print(mysklearn.myevaluation.metrics.fmeasure(bayes_predict, predict_actual))

0.8656355555555555
0.1306603478266607
0.14336957558972066
0.015191137912377016


In [22]:
cmat = myevaluation.confusion_matrix(bayes_predict, predict_actual, labels)
cmat_table = MyPyTable(data=cmat, column_names=labels)

In [23]:
cmat_table.pretty_print()

  1940    1950    1960    1970    1980    1990    2000    2010    2020
------  ------  ------  ------  ------  ------  ------  ------  ------
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
   384    1001    2380    2721    6199   45759   72894   69623    2135
  1109    1428    3354    5010    6538   36918   94622   77066    2278
   128     273    1654    5251    9805   53659  206548  253161   11939
     0       1       3      24     134    3244    9752   11187    1818


In [24]:
rand = MyRandomClassifier()
zeroR = MyZeroRClassifier()

In [25]:
rand.fit(x_data, y_data)
zeroR.fit(x_data, y_data)

In [26]:
predict_out = rand.predict(predict_test)
print(mysklearn.myevaluation.metrics.accuracy(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.precision(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.recall(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.fmeasure(predict_out, predict_actual))
cmat = myevaluation.confusion_matrix(predict_out, predict_actual, labels)
cmat_table = MyPyTable(data=cmat, column_names=labels)
cmat_table.pretty_print()

0.8500222222222221
0.11112534246845193
0.11097117918406965
0.012338689702419975
  1940    1950    1960    1970    1980    1990    2000    2010    2020
------  ------  ------  ------  ------  ------  ------  ------  ------
     3       3       7      14      33     188     564     592      23
     5       7      17      32      61     385     971    1074      41
     7      22      44      97     158    1017    2761    3017     128
    19      45     103     211     331    2032    5358    5703     233
    28      57     192     309     521    3283    9047    9634     463
   199     327     929    1608    2911   17652   48976   52053    2287
   598     977    2602    4490    8014   49273  135499  144992    6397
   665    1113    3071    5473    9366   57619  158408  170151    7586
    97     152     426     772    1281    8131   22232   23821    1012


In [27]:
predict_out = zeroR.predict(predict_test)
print(mysklearn.myevaluation.metrics.accuracy(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.precision(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.recall(predict_out, predict_actual))
print(mysklearn.myevaluation.metrics.fmeasure(predict_out, predict_actual))
cmat = myevaluation.confusion_matrix(predict_out, predict_actual, labels)
cmat_table = MyPyTable(data=cmat, column_names=labels)
cmat_table.pretty_print()

0.8691193333333334
0.045670777777777775
0.1111111111111111
0.007192626223405241
  1940    1950    1960    1970    1980    1990    2000    2010    2020
------  ------  ------  ------  ------  ------  ------  ------  ------
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
     0       0       0       0       0       0       0       0       0
  1621    2703    7391   13006   22676  139580  383816  411037   18170
     0       0       0       0       0       0       0       0       0
